# Loading Parameters
The network output is different when running on GPU vs. CPU, presumably due to formatting differences (NCHW vs. NHWC). Is this the case, and if so, can it be fixed?

## Initial setup

In [1]:
import sys
sys.path.insert(0, "../python")
from vizdoom import *
import tensorflow as tf
from helper import create_agent
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np

In [3]:
# Initializes DoomGame from config file
def initialize_vizdoom(config_file):
    game = DoomGame()
    game.load_config(config_file)
    game.init()
    return game  

In [4]:
agent_file_path = "../agents/ddqn_dm2015_1.json"
config_file_path = "../config/linear_track.cfg"
results_dir = "./loading_params/results_dir"
action_set = "basic_four"
params_file_path = "../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100"

## CPU results
By setting the environment variable `CUDA_VISIBLE_DEVICES` to empty, we force TensorFlow to load everything on the CPU. The trained network on the GPU had output Q values for the initial state around -4.5 for all actions.

In [20]:
os.environ["CUDA_VISIBLE_DEVICES"]=""
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=False)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("data format: ", agent.network.data_format)
print("state shape: ", agent.state.shape)
print("Q: ", agent.network.get_q_values(agent.state))

INFO:tensorflow:Restoring parameters from ../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100
data format:  NHWC
state shape:  (84, 84, 12)
Q:  [[-21.41444397 -23.56121445 -21.40477943 -19.81874275]]


Clearly, these values are off. What are the shapes of the variables and layers in this NHWC format?

In [21]:
[print(v) for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="main_network") \
 if "CONV" in v.name]
print(tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/Relu:0"))

<tf.Variable 'main_network/CONV_1/weights:0' shape=(8, 8, 12, 32) dtype=float32_ref>
<tf.Variable 'main_network/CONV_1/biases:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'main_network/CONV_2/weights:0' shape=(4, 4, 32, 64) dtype=float32_ref>
<tf.Variable 'main_network/CONV_2/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/weights/RMSProp:0' shape=(8, 8, 12, 32) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/weights/RMSProp_1:0' shape=(8, 8, 12, 32) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/biases/RMSProp:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/biases/RMSProp_1:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_2/weights/RMSProp:0' shape=(4, 4, 32, 64) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_2/weights/RMSProp_1:0' shape=(4, 4, 32, 64) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_2/biases/RMSP

Okay, does creating the network without loading parameters change anything?

In [22]:
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=None,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=False)
[print(v) for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="main_network") \
 if "CONV" in v.name]
print(tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/Relu:0"))

<tf.Variable 'main_network/CONV_1/weights:0' shape=(8, 8, 12, 32) dtype=float32_ref>
<tf.Variable 'main_network/CONV_1/biases:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'main_network/CONV_2/weights:0' shape=(4, 4, 32, 64) dtype=float32_ref>
<tf.Variable 'main_network/CONV_2/biases:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/weights/RMSProp:0' shape=(8, 8, 12, 32) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/weights/RMSProp_1:0' shape=(8, 8, 12, 32) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/biases/RMSProp:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_1/biases/RMSProp_1:0' shape=(32,) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_2/weights/RMSProp:0' shape=(4, 4, 32, 64) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_2/weights/RMSProp_1:0' shape=(4, 4, 32, 64) dtype=float32_ref>
<tf.Variable 'main_network/main_network/CONV_2/biases/RMSP

No, the shapes are the same. What are the values of the weights and intermediate layers?

In [33]:
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=False)
agent.initialize_new_episode()
sess = agent.network.sess
s1 = agent.network._check_state(agent.state)
fd = {agent.network.state: s1}
conv1_w = tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/weights:0")
print("CONV_1 weights: ", sess.run(conv1_w[:, :, 0, 0]))
conv1_b = tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/biases:0")
print("CONV_1 biases: ", sess.run(conv1_b[:5]))
conv2_w = tf.get_default_graph().get_tensor_by_name("main_network/CONV_2/weights:0")
print("CONV_2 weights: ", sess.run(conv2_w[:, :, 0, 0]))
conv2_b = tf.get_default_graph().get_tensor_by_name("main_network/CONV_2/biases:0")
print("CONV_2 biases: ", sess.run(conv2_b[:5]))
conv1_relu = tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/Relu:0")
print("CONV_1 output: ", sess.run(conv1_relu, feed_dict=fd)[0, :5, :5, 10])

INFO:tensorflow:Restoring parameters from ../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100
CONV_1 weights:  [[-0.21754628 -0.02670558  0.17301919 -0.21873856 -0.04389495 -0.21369149
   0.33146971 -0.16915736]
 [ 0.09168395  0.23878217  0.2326481   0.02665762 -0.2114628  -0.03435934
   0.07497527 -0.26659325]
 [-0.25824946  0.06228131  0.14520782 -0.00580286 -0.28452948 -0.30995023
  -0.14012729 -0.19221263]
 [-0.12438106  0.06364045  0.25386235 -0.09242661 -0.24531648 -0.28857112
  -0.04376693 -0.07326075]
 [ 0.0009142   0.05994602  0.0938239  -0.1217043  -0.2188206  -0.12539618
   0.04423197 -0.15963058]
 [ 0.06693146  0.05401253 -0.17208847  0.02707538 -0.20046464  0.02171924
   0.11382003 -0.08820896]
 [ 0.02807241  0.02692594 -0.14877053 -0.15393072 -0.2587145   0.0991367
   0.18433155 -0.17461354]
 [-0.1480585  -0.24462163 -0.24611878 -0.00386733 -0.34481272  0.0156329
   0.05573718 -0.14490856]]
CONV_1 biases:  [-0.06518342  0.07556296  0

## GPU Results
Let's compare the values above to those on the GPU. We first must restart the kernel and set the environment variable prior to importing tensorflow.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# Now run initial setup cells

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [7]:
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=False)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("data format: ", agent.network.data_format)
print("state shape: ", agent.state.shape)
print("Q: ", agent.network.get_q_values(agent.state))

INFO:tensorflow:Restoring parameters from ../experiments/linear_track_1/trial_20/train_data/net_data/main_net/params/train_model-100


ResourceExhaustedError: OOM when allocating tensor with shape[4,4,32,64]
	 [[Node: main_network/save/Assign_3 = Assign[T=DT_FLOAT, _class=["loc:@main_network/CONV_2/weights"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](main_network/CONV_2/weights, main_network/save/RestoreV2_3/_43)]]

Caused by op 'main_network/save/Assign_3', defined at:
  File "/home/james/anaconda3/envs/vizdoom/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/james/anaconda3/envs/vizdoom/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/james/anaconda3/envs/vizdoom/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/james/anaconda3/envs/vizdoom/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/james/anaconda3/envs/vizdoom/lib/python3.6/site-packages/ip

In [ ]:
[print(v) for v in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="main_network") \
 if "CONV" in v.name]
print(tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/Relu:0"))

In [ ]:
sess = agent.network.sess
s1 = agent.network._check_state(agent.state)
fd = {agent.network.state: s1}
conv1_w = tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/weights:0")
print("CONV_1 weights: ", sess.run(conv1_w[:, :, 0, 0]))
conv1_b = tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/biases:0")
print("CONV_1 biases: ", sess.run(conv1_b[:5]))
conv2_w = tf.get_default_graph().get_tensor_by_name("main_network/CONV_2/weights:0")
print("CONV_2 weights: ", sess.run(conv2_w[:, :, 0, 0]))
conv2_b = tf.get_default_graph().get_tensor_by_name("main_network/CONV_2/biases:0")
print("CONV_2 biases: ", sess.run(conv2_b[:5]))
conv1_relu = tf.get_default_graph().get_tensor_by_name("main_network/CONV_1/Relu:0")
print("CONV_1 output: ", sess.run(conv1_relu, feed_dict=fd)[0, :5, :5, 10])